In [1]:
# Initial imports
import numpy as np
import pandas as pd


# API import
import requests
import json

# Data cleaning
from datetime import datetime


# Plot import
%matplotlib inline

In [26]:
# API
ticker = "BTC"
limit ="2000"

price_url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={ticker}&tsym=USD&limit=2000"

price_response_data = requests.get(price_url).json()
data_price = pd.DataFrame(price_response_data["Data"]["Data"])

#volume_url = "https://min-api.cryptocompare.com/data/symbol/histoday?fsym=ETH&tsym=USD&limit=2000"
#volume_response_data = requests.get(volume_url).json()

#data_vol = pd.DataFrame(volume_response_data)

data_price.head()


,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1469750400,657.40,650.99,654.13,27043.62,17757920.48,655.43,direct,
1,1469836800,657.55,652.06,655.43,19354.41,12721755.54,654.74,direct,
2,1469923200,654.92,621.42,654.74,62887.74,39918066.14,621.87,direct,
3,1470009600,627.89,603.55,621.87,66660.27,41229298.83,607.00,direct,
4,1470096000,613.75,471.36,607.00,130913.66,75338595.54,513.43,direct,


In [27]:
# Transform to DataFrame
data_price.drop(columns = ["volumefrom" , "volumeto" , "conversionType" , "conversionSymbol"], inplace =True)

data_price.columns =['Date', 'Highest price',  'Lowest price' ,  'Opening price' , 'Closing price']
data_price['Date'] = pd.to_datetime(data_price['Date'], unit='s')

data_price.set_index('Date', inplace=True)

data_price.head()

,Highest price,Lowest price,Opening price,Closing price
Date,,,,
2016-07-29,657.40,650.99,654.13,655.43
2016-07-30,657.55,652.06,655.43,654.74
2016-07-31,654.92,621.42,654.74,621.87
2016-08-01,627.89,603.55,621.87,607.00
2016-08-02,613.75,471.36,607.00,513.43


In [54]:
# Select only dates starting end 2019, because the price curve is distincly different
df = data_price.loc["2019-12-1" : ]
df.head(10)

,Highest price,Lowest price,Opening price,Closing price
Date,,,,
2019-12-01,7575.50,7261.95,7571.77,7420.53
2019-12-02,7442.62,7193.09,7420.53,7320.94
2019-12-03,7420.34,7260.93,7320.94,7313.64
2019-12-04,7764.68,7110.76,7313.64,7207.78
2019-12-05,7492.96,7175.85,7207.78,7406.75
2019-12-06,7607.79,7323.86,7406.75,7558.58
2019-12-07,7640.51,7503.10,7558.58,7514.86
2019-12-08,7594.80,7414.51,7514.86,7539.70
2019-12-09,7667.04,7296.42,7539.70,7350.62


In [55]:
def window_data(df, window, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    for i in range(len(df) - window -1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window + 1), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

# Split Data and create predictive set

In [56]:
# Define the window size
window_size = 10


# Set the index of the feature and target columns
feature_column = 3
target_column = 3

# Create the features (X) and target (y) data using the window_data() function.
X, y = window_data(df, window_size, feature_column, target_column)


# Manually splitting the data
split = int(0.8 * len(X))

X_train = X[: split]
X_test = X[split:]

y_train = y[: split]
y_test = y[split:]

In [57]:
# Importing the MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

# Fit the MinMaxScaler object with the features data X
X_scaler.fit(X_train)

# Scale the features training and testing sets
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Fit the MinMaxScaler object with the target data Y
Y_scaler.fit(y_train)

# Scale the target training and testing sets
Y_scaler = scaler.transform(y_train)
Y_scaler = scaler.transform(y_test)

In [58]:
# Reshape the features data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build and Train the LSTM RNN

In [59]:
# Import required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [60]:
model = Sequential()

# Initial model setup
number_units = 20
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))


# Output layer
model.add(Dense(1))

In [61]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 10, 20)            1760      
                                                                 
 dropout_11 (Dropout)        (None, 10, 20)            0         
                                                                 
 lstm_12 (LSTM)              (None, 10, 20)            3280      
                                                                 
 dropout_12 (Dropout)        (None, 10, 20)            0         
                                                                 
 dense_5 (Dense)             (None, 10, 1)             21        
                                                                 
Total params: 5,061
Trainable params: 5,061
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error" , metrics = ["mse", "accuracy"])

In [63]:
# Train the model
model.fit(X_train,
          y_train,
          epochs=12,
          #validation_split=0.2,
          shuffle=False,
          batch_size=1,
          verbose=1
)

Epoch 1/12
616/616 [==============================] - 5s 4ms/step - loss: 849157248.0000 - mse: 849157248.0000 - accuracy: 0.0000e+00
Epoch 2/12
616/616 [==============================] - 3s 4ms/step - loss: 848558400.0000 - mse: 848558400.0000 - accuracy: 0.0000e+00
Epoch 3/12
616/616 [==============================] - 3s 4ms/step - loss: 848078848.0000 - mse: 848078848.0000 - accuracy: 0.0000e+00
Epoch 4/12
616/616 [==============================] - 3s 4ms/step - loss: 847605056.0000 - mse: 847605056.0000 - accuracy: 0.0000e+00
Epoch 5/12
616/616 [==============================] - 2s 4ms/step - loss: 847136000.0000 - mse: 847136000.0000 - accuracy: 0.0000e+00
Epoch 6/12
616/616 [==============================] - 2s 4ms/step - loss: 846671360.0000 - mse: 846671360.0000 - accuracy: 0.0000e+00
Epoch 7/12
616/616 [==============================] - 3s 4ms/step - loss: 846213952.0000 - mse: 846213952.0000 - accuracy: 0.0000e+00
Epoch 8/12
616/616 [==============================] - 3s 4ms/s

In [51]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 4ms/step - loss: 2721379328.0000 - mse: 2721379328.0000 - accuracy: 0.0000e+00


[2721379328.0, 2721379328.0, 0.0]

In [52]:
# Make some predictions
predicted = model.predict(X_test)

# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Actual": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = data_price.index[-len(real_prices): ]) 

# Show the DataFrame's head
stocks.tail()

In [ ]:
stocks.plot(title=f"Actual Vs. Predicted {ticker} Prices");